In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os

import excursion.testcases.fast3d as scandetails
import excursion.optimize
import datetime
import matplotlib as plt
import numpy as np

import excursion.plotting.twodim as plots
import excursion.utils as utils

np.warnings.filterwarnings('ignore')

C:\Users\Tutrie\anaconda3\envs\IRIS_excursion\lib\site-packages\numpy\lib\index_tricks.py:183: ComplexWarning: Casting complex values to real discards the imaginary part
  nn[k] = (nn[k]*step+start)


In [3]:

grid = ix,iy = scandetails.plotG
X = utils.mesh2points(grid,scandetails.plot_rangedef[:,2])

functions = [
    scandetails.truth,
    lambda X: scandetails.truth(X-0.05)
]


scandetails.truth_functions = functions


truthv = excursion.utils.values2mesh(
    scandetails.truth(X),
    scandetails.plot_rangedef,
    scandetails.invalid_region
)

contours = []
for truth_func in scandetails.truth_functions:
    contours.append(
        excursion.utils.values2mesh(
            truth_func(X),
            scandetails.plot_rangedef,
            scandetails.invalid_region
        )
    )


for cont,c in zip(contours, ['k','b']):
    plt.contour(ix,iy,cont, levels = [0], colors=[c])

ValueError: too many values to unpack (expected 2)

In [ ]:
#initialize it...
N_INIT  = 5
N_FUNCS = 2

np.random.seed(1)
X = np.random.uniform(scandetails.plot_rangedef[:,0],scandetails.plot_rangedef[:,1], size = (N_INIT,2))
y_list = [np.array([scandetails.truth_functions[i](np.asarray([x]))[0] for x in X]) for i in range(N_FUNCS)]

In [ ]:
gps = [excursion.get_gp(X,y_list[i]) for i in range(N_FUNCS)]
fig, axarr = plt.subplots(1, 3, sharey=True)
fig.set_size_inches(15, 4.5)
plots.plot(axarr,gps,X,y_list,scandetails)
# plt.savefig(os.path.join(workdir,'update_{}.png'.format(str(index).zfill(3))))
plt.show()

In [ ]:
import time

N_UPDATES = 50

workdir = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S-2dmulti')
os.mkdir(workdir)

for index in range(N_UPDATES):
    start = time.process_time()
    gps = [excursion.get_gp(X,y_list[i]) for i in range(N_FUNCS)]

    fig, axarr = plt.subplots(1, 3, sharey=True)
    fig.set_size_inches(4.5, 1.5)
    plots.plot(axarr,gps,X,y_list,scandetails)
    plt.savefig(os.path.join(workdir,'update_{}.png'.format(str(index).zfill(3))))
    plt.show()

    newx, acqvals = excursion.optimize.gridsearch(gps, X, scandetails)
    print('TIME TAKEN**', time.process_time() - start)
    print('new x: {}'.format(newx))
    newy_list = [scandetails.truth_functions[i](np.asarray([newx]))[0] for i in range(N_FUNCS)]
    for i,newy in enumerate(newy_list):
        print('new y i: {} {}'.format(i,newy))
        y_list[i] = np.concatenate([y_list[i],[newy]])
    X = np.concatenate([X,np.array([newx])])




In [ ]:
scandetails.acqX.shape

In [ ]:
41**2